In [16]:
%matplotlib widget

In [17]:
from glob import glob
import numpy as np
import pandas as pd
import flammkuchen as fl
from split_dataset import SplitDataset
from bouter import Experiment
from motions.utilities import stim_vel_dir_dataframe, quantize_directions
from scipy.interpolate import interp1d 
from scipy.signal import convolve2d
import colorspacious
import matplotlib.pyplot as plt

from fimpylab.core.lightsheet_experiment import LightsheetExperiment

import json
from pathlib import Path

from scipy import stats

In [3]:
master_ntr = Path(r"\\portulab.synology.me\data\Hagar and Ot\E0040\v10\LS ablation\ntr")
master_control = Path(r"\\portulab.synology.me\data\Hagar and Ot\E0040\v10\LS ablation\control")

pre_list = list(master_ntr.glob("*_ntr_pre*"))
pre_list = [f for f in pre_list if (f / "data_from_suite2p_cells.h5").exists()]

post_list = list(master_ntr.glob("*_ntr_post*"))
post_list = [f for f in post_list if (f / "data_from_suite2p_cells.h5").exists()]


pre_list_ctrl = list(master_control.glob("*_control_pre*"))
pre_list_ctrl = [f for f in pre_list_ctrl if (f / "data_from_suite2p_cells.h5").exists()]

post_list_ctrl = list(master_control.glob("*_control_post*"))
post_list_ctrl = [f for f in post_list_ctrl if (f / "data_from_suite2p_cells.h5").exists()]

num_fish_ntr = len(pre_list)
num_fish_ctrl = len(pre_list)

num_fish_ntr = len(pre_list)

print(num_fish_ntr)

11


In [4]:
regions = ['ipn_coords', 'ahb_coords']
num_regions = len(regions)
thresh = 0.3
n_tuned_pre = np.zeros((num_regions, num_fish_ntr)) # IPN, aHB
n_neurons_pre = np.zeros((num_regions, num_fish_ntr)) # IPN, aHB
n_tuned_post = np.zeros((num_regions, num_fish_ntr)) # IPN, aHB
n_neurons_post = np.zeros((num_regions, num_fish_ntr)) # IPN, aHB

In [5]:
bins = np.linspace(-1, 1, 100)    
fig2, ax2 = plt.subplots(num_regions, num_fish_ntr, figsize=(12, 7))

for fish in range(num_fish_ntr):
    pre_fish = pre_list[fish]
    exp = Experiment(pre_fish)
    fish_id = exp.fish_id
    print(fish_id)
    
    coords_regions = fl.load(pre_fish / "regions_masks_mapzbrain.h5")
    rel_ind_all = fl.load(pre_fish / "reliability_index_arr.h5")['reliability_arr_combined']
    
    for region in range(num_regions):
        
        # getting region coords 
        region_idx = coords_regions[regions[region]]
        num_traces = np.shape(region_idx)[0]
        rel_ind = rel_ind_all[region_idx]

        n_neurons_pre[region, fish] = num_traces
        n_tuned_pre[region, fish] = len(np.where(rel_ind > thresh)[0])
        ax2[region, fish].hist(rel_ind, bins, alpha=0.5, label='pre', density=False)
        ax2[region, 0].set_ylabel("Count(" + regions[region] + ")")
    
    ax2[0, fish].set_title(fish_id)
    ax2[0, fish].set_xlabel("Correlation")
    
    #######################################
    post_fish = post_list[fish]
    
    exp = Experiment(post_fish)
    fish_id = exp.fish_id
    print(fish_id)

    coords_regions = fl.load(post_fish / "regions_masks_mapzbrain.h5")
    rel_ind_all = fl.load(post_fish / "reliability_index_arr.h5")['reliability_arr_combined']
    
    for region in range(num_regions):
        
        # getting region coords 
        region_idx = coords_regions[regions[region]]
        num_traces = np.shape(region_idx)[0]
        rel_ind = rel_ind_all[region_idx]

        n_neurons_post[region, fish] = num_traces
        n_tuned_post[region, fish] = len(np.where(rel_ind > thresh)[0])
        ax2[region, fish].hist(rel_ind, bins, alpha=0.5, label='post', density=False)
        ax2[0, num_fish_ntr-1].legend(loc='upper right')
        
        ax2[region, fish].spines['right'].set_visible(False)
        ax2[region, fish].spines['top'].set_visible(False)
   

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

240708_f0
240709_f0


No handles with labels found to put in legend.
No handles with labels found to put in legend.


240708_f1
240710_f1


No handles with labels found to put in legend.
No handles with labels found to put in legend.


...

240709_f6
240711_f6


No handles with labels found to put in legend.
No handles with labels found to put in legend.


240709_f7
240711_f7


No handles with labels found to put in legend.
No handles with labels found to put in legend.


240709_f8
240711_f8


In [6]:
plt.subplots_adjust(left=0.05, right=0.95, wspace=0.2)

In [7]:
for i in range(num_fish_ntr):
    ax2[0, i].set_xlim(-0.2,1)
    ax2[1, i].set_xlim(-0.2,1)

In [8]:
n_tuned_pre_ctrl = np.zeros((num_regions, num_fish_ctrl)) # IPN, aHB
n_neurons_pre_ctrl = np.zeros((num_regions, num_fish_ctrl)) # IPN, aHB
n_tuned_post_ctrl = np.zeros((num_regions, num_fish_ctrl)) # IPN, aHB
n_neurons_post_ctrl = np.zeros((num_regions, num_fish_ctrl)) # IPN, aHB

In [9]:
bins = np.linspace(-1, 1, 100)    
fig2, ax2 = plt.subplots(num_regions, num_fish_ctrl, figsize=(12, 7))

for fish in range(num_fish_ctrl):
    pre_fish = pre_list_ctrl[fish]
    exp = Experiment(pre_fish)
    fish_id = exp.fish_id
    print(fish_id)
    
    coords_regions = fl.load(pre_fish / "regions_masks_mapzbrain.h5")
    rel_ind_all = fl.load(pre_fish / "reliability_index_arr.h5")['reliability_arr_combined']
    
    for region in range(num_regions):
        
        # getting region coords 
        region_idx = coords_regions[regions[region]]
        num_traces = np.shape(region_idx)[0]
        rel_ind = rel_ind_all[region_idx]

        n_neurons_pre_ctrl[region, fish] = num_traces
        n_tuned_pre_ctrl[region, fish] = len(np.where(rel_ind > thresh)[0])
        ax2[region, fish].hist(rel_ind, bins, alpha=0.5, label='pre', density=False)
        ax2[region, 0].set_ylabel("Count(" + regions[region] + ")")
    
    ax2[0, fish].set_title(fish_id)
    ax2[0, fish].set_xlabel("Correlation")
    
    #######################################
    post_fish = post_list_ctrl[fish]
    
    exp = Experiment(post_fish)
    fish_id = exp.fish_id
    print(fish_id)
    
    coords_regions = fl.load(post_fish / "regions_masks_mapzbrain.h5")
    rel_ind_all = fl.load(post_fish / "reliability_index_arr.h5")['reliability_arr_combined']
    
    for region in range(num_regions):
        
        # getting region coords 
        region_idx = coords_regions[regions[region]]
        num_traces = np.shape(region_idx)[0]
        rel_ind = rel_ind_all[region_idx]

        n_neurons_post_ctrl[region, fish] = num_traces
        n_tuned_post_ctrl[region, fish] = len(np.where(rel_ind > thresh)[0])
        ax2[region, fish].hist(rel_ind, bins, alpha=0.5, label='post', density=False)
        ax2[0, num_fish_ctrl-1].legend(loc='upper right')
        
        ax2[region, fish].spines['right'].set_visible(False)
        ax2[region, fish].spines['top'].set_visible(False)
   

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

240708_f10
240710_f10


No handles with labels found to put in legend.
No handles with labels found to put in legend.


240708_f5
240710_f5


No handles with labels found to put in legend.
No handles with labels found to put in legend.


...

240709_f3
240711_f3


No handles with labels found to put in legend.
No handles with labels found to put in legend.


240709_f4
240711_f4


No handles with labels found to put in legend.
No handles with labels found to put in legend.


240709_f9
240711_f9


In [10]:
#fig2.savefig(master / 'reliability index distribution.pdf', dpi=300)

In [11]:
num_corr_cells = np.append(n_tuned_pre, n_tuned_post, axis=0)
num_corr_cells_ipn = num_corr_cells[[0,2],:]
num_corr_cells_ahb = num_corr_cells[[1,3],:]

In [39]:
fig1, ax1 = plt.subplots(2,2, figsize=(5,4))

ax1[0,0].plot(num_corr_cells_ipn, color='gray')
ax1[0,0].spines['right'].set_visible(False)
ax1[0,0].spines['top'].set_visible(False)
ax1[0,0].set_xlim(-0.5, 1.5)

ax1[0,1].plot(num_corr_cells_ahb, color='gray')
ax1[0,1].spines['right'].set_visible(False)
ax1[0,1].spines['top'].set_visible(False)
ax1[0,1].set_xlim(-0.5, 1.5)
ax1[0,0].set_ylabel('Cell count (IPN)')
ax1[0,1].set_ylabel('Cell count (aHB)')

fig1.subplots_adjust(left=0.15, right=0.95, wspace=0.6)
ax1[0,0].set_ylim(0, 100)
ax1[1,0].set_ylim(0, 100)
ax1[0,1].set_ylim(0, 900)
ax1[1,1].set_ylim(0, 900)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(0.0, 900.0)

In [40]:
ax1[0,0].errorbar([0, 1], np.nanmean(num_corr_cells_ipn, axis=1), np.nanstd(num_corr_cells_ipn, axis=1) / 11, color='k')
ax1[0,1].errorbar([0, 1], np.nanmean(num_corr_cells_ahb, axis=1), np.nanstd(num_corr_cells_ahb, axis=1) / 11, color='k')

#ax1[0,0].scatter([0, 1], np.nanmean(num_corr_cells_ipn, axis=1), color='k')
#ax1[0,0].scatter([0, 0], np.nanmean(num_corr_cells_ahb, axis=1), color='k')

<ErrorbarContainer object of 3 artists>

In [27]:
np.nanmean(num_corr_cells_ipn, axis=1)

array([23.18181818, 11.09090909])

In [23]:
num_corr_cells_ctrl = np.append(n_tuned_pre_ctrl, n_tuned_post_ctrl, axis=0)
num_corr_cells_ipn_ctrl = num_corr_cells_ctrl[[0,2],:]
num_corr_cells_ahb_ctrl = num_corr_cells_ctrl[[1,3],:]

In [41]:
ax1[1,0].plot(num_corr_cells_ipn_ctrl, color='gray')
ax1[1,0].spines['right'].set_visible(False)
ax1[1,0].spines['top'].set_visible(False)
ax1[1,0].set_xlim(-0.5, 1.5)

ax1[1,1].plot(num_corr_cells_ahb_ctrl, color='gray')
ax1[1,1].spines['right'].set_visible(False)
ax1[1,1].spines['top'].set_visible(False)
ax1[1,1].set_xlim(-0.5, 1.5)
ax1[1,0].set_ylabel('Cell count (IPN)')
ax1[1,1].set_ylabel('Cell count (aHB)')

Text(349.34508547008545, 0.5, 'Cell count (aHB)')

In [42]:
ax1[1,0].errorbar([0, 1], np.nanmean(num_corr_cells_ipn_ctrl, axis=1), np.nanstd(num_corr_cells_ipn_ctrl, axis=1) / 11, color='k')
ax1[1,1].errorbar([0, 1], np.nanmean(num_corr_cells_ahb_ctrl, axis=1), np.nanstd(num_corr_cells_ahb_ctrl, axis=1) / 11, color='k')

<ErrorbarContainer object of 3 artists>

In [43]:
fig1.savefig(master_ntr / 'cell count change thresh 0.3.pdf', dpi=300)

In [ ]:
############### t-test for ntr group
stats.ttest_rel(num_corr_cells_ipn[0], num_corr_cells_ipn[1], alternative='greater')

In [ ]:
############### t-test for ntr group
stats.ttest_rel(num_corr_cells_ahb[0], num_corr_cells_ahb[1], alternative='greater')

In [ ]:
############### t-test for control group
stats.ttest_rel(num_corr_cells_ipn_ctrl[0], num_corr_cells_ipn_ctrl[1], alternative='greater')

In [ ]:
############### t-test for control group
stats.ttest_rel(num_corr_cells_ahb_ctrl[0], num_corr_cells_ahb_ctrl[1], alternative='greater')

In [ ]:
############### same as before but fraction of neurons instead of neuron count 

In [ ]:
fraction_corr_pre = n_tuned_pre / n_neurons_pre 
fraction_corr_post = n_tuned_post / n_neurons_post

fraction_corr_cells = np.append(fraction_corr_pre, fraction_corr_post, axis=0)
fraction_corr_cells_ipn = fraction_corr_cells[[0,2],:]
fraction_corr_cells_ahb = fraction_corr_cells[[1,3],:]

In [ ]:
fig2, ax2 = plt.subplots(2,2, figsize=(5,4), sharex=True, sharey=True)

ax2[0,0].plot(fraction_corr_cells_ipn, color='gray')
ax2[0,0].spines['right'].set_visible(False)
ax2[0,0].spines['top'].set_visible(False)
ax2[0,0].set_xlim(-0.5, 1.5)

ax2[0,1].plot(fraction_corr_cells_ahb, color='gray')
ax2[0,1].spines['right'].set_visible(False)
ax2[0,1].spines['top'].set_visible(False)
ax2[0,1].set_xlim(-0.5, 1.5)
ax2[0,0].set_ylabel('% Tuned cells (IPN)')
ax2[0,1].set_ylabel('% Tuned cells (aHB)')


ax1[0,0].set_ylim(0, 1)
ax1[0,0].set_xlim(0, 1)


In [ ]:
fraction_corr_pre_ctrl = n_tuned_pre_ctrl / n_neurons_pre_ctrl
fraction_corr_post_ctrl = n_tuned_post_ctrl / n_neurons_post_ctrl

fraction_corr_cells_ctrl = np.append(fraction_corr_pre_ctrl, fraction_corr_post_ctrl, axis=0)
fraction_corr_cells_ipn_ctrl = fraction_corr_cells_ctrl[[0,2],:]
fraction_corr_cells_ahb_ctrl = fraction_corr_cells_ctrl[[1,3],:]

In [ ]:
ax2[1,0].plot(fraction_corr_cells_ipn_ctrl, color='gray')
ax2[1,0].spines['right'].set_visible(False)
ax2[1,0].spines['top'].set_visible(False)
ax2[1,0].set_xlim(-0.5, 1.5)

ax2[1,1].plot(fraction_corr_cells_ahb_ctrl, color='gray')
ax2[1,1].spines['right'].set_visible(False)
ax2[1,1].spines['top'].set_visible(False)
ax2[1,1].set_xlim(-0.5, 1.5)
ax2[1,0].set_ylabel('% Tuned cells (IPN)')
ax2[1,1].set_ylabel('% Tuned cells (aHB)')

In [ ]:
############### t-test for ntr group
print(stats.ttest_rel(num_corr_cells_ipn[0], num_corr_cells_ipn[1], alternative='greater'))
############### t-test for ntr group
print(stats.ttest_rel(num_corr_cells_ahb[0], num_corr_cells_ahb[1], alternative='greater'))
############### t-test for control group
print(stats.ttest_rel(num_corr_cells_ipn_ctrl[0], num_corr_cells_ipn_ctrl[1], alternative='greater'))
############### t-test for control group
print(stats.ttest_rel(num_corr_cells_ahb_ctrl[0], num_corr_cells_ahb_ctrl[1], alternative='greater'))